# WEFE Site Analyst Tool
## implemented in Jupyter Notebook and applied for a Case Study in Costa Rica

This Jupyter Notebook collects environmental and socioeconomic data for the planning of integrated water, energy, food, and environment systems from open servers.

In [17]:
# imports
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import os
from math import sqrt
import numpy as np
#import folium
#import geojson
import geemap
import geemap.foliumap as gfolium
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
from geojson import Point, Feature, FeatureCollection, dump

from shapely.geometry import Polygon, shape

# import files from source
os.chdir("../../src/")
import era5
import ee_layer
os.chdir("../examples/Costa Rica")

In [5]:
# provide case study name
name = 'finca_turrialba'
# provide input in form of coordinates
lon = -83.616490
lat = 9.830153

In [6]:
# set start and end date (end date will be included
# in the time period for which data is downloaded)
start_date, end_date = '2022-05-30', '2022-11-30'  # time in UTC choose start date one day before time of interest
# for position east of 0° meridian for covering all hours of interest

In [ ]:
# initiate earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [4]:

# visualization as geemap
map = geemap.Map(center=[lat, lon], zoom=17)
map.add_basemap('SATELLITE')
# map.add_basemap('TERRAIN')

print("please draw center of demand and area of interest into the map displayed below")

# open aoi geojson file

file_path = os.path.abspath('aoi.geojson')
with open(file_path) as f:
    aoi_geojson = json.load(f)
print(aoi_geojson)

# geojson to ee
# aoi = geojson_to_ee(aoi_geojson)

# Create ipiLEAFlet layer from the GeoJSONs    
json_shape_layer = GeoJSON(
    data=aoi_geojson,
    name='AOI',
    hover_style={'fillColor': 'red', 'fillOpacity': 0.1},
)

map.add_layer(json_shape_layer)
map


Enter verification code: 4/1AWtgzh7LsGlD0Ahl3GzDzsBZ1j8nQzZS1T20EAuQBE9X3CkJz7PfeRPqX98

Successfully saved authorization token.
please draw center of demand and area of interest into the map displayed below
{'type': 'FeatureCollection', 'features': {'type': 'FeatureCollection', 'columns': {'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-83.616471, 9.830277], [-83.616503, 9.830488], [-83.616417, 9.830657], [-83.616353, 9.830974], [-83.61616, 9.831492], [-83.61602, 9.831757], [-83.616128, 9.831915], [-83.616578, 9.831947], [-83.616836, 9.831936], [-83.617029, 9.831957], [-83.617254, 9.831979], [-83.617512, 9.831513], [-83.617855, 9.831482], [-83.618381, 9.831365], [-83.618553, 9.831218], [-83.618756, 9.830552], [-83.618799, 9.829547], [-83.618606, 9.829357], [-83.617898, 9.82904], [-83.617683, 9.828924], [-83.617179, 9.828649], [-83.616943, 9.828585], [-83.616675, 9.828543], [-83.616449, 9.828511], [-83.615913, 9.828458], [-

Map(center=[9.830153, -83.61649], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [ ]:
map.draw_last_feature


In [ ]:
# retrieve area of intrest from drawing

aoi = ee.FeatureCollection(map.draw_last_feature)

# convert to geoJSON and save in folder

aoi_geojson = ee_to_geojson(aoi)


feature_collection = FeatureCollection(aoi_geojson)

with open('aoi.geojson', 'w') as f:
   dump(feature_collection, f)


In [ ]:

# create GeoJSON for the specific point
center = {
  "type": "Point",
  "coordinates": [lon, lat]
}

# file_path = os.path.abspath('pp_geometry.geojson')
# with open(file_path) as f:
#    polygon_json = json.load(f)
    

# Calculation of project area
# Convert the GeoJSON to a Shapely polygon
# polygon_shapely = shape(polygon_json)

# Calculate the area of the polygon
# area = polygon.area

# print(area)
    
    
# Create ipiLEAFlet layer from the GeoJSONs    
#json_shape_layer = GeoJSON(
#    data=polygon_json,
#    name='target area',
#    hover_style={'fillColor': 'red', 'fillOpacity': 0.1},
#)
json_center =GeoJSON(data=center,
                    name='center'
)

# adding shape layers to geemap
# map.add_layer(json_shape_layer)
map.add_layer(json_center)

In [ ]:
## TERAIN MAP
# Define the location for which you want to create the elevation map (given abo)

center = ee.Geometry.Point(lon, lat)
buffer = center.buffer(3000)

# Get the elevation at the map center
elevation = ee.Image('USGS/SRTMGL1_003').sample(center)

# Get the elevation value at the map center
elevation_value = elevation.get('elevation').getInfo()

# Print the elevation value
print(f'Elevation at {lon}, {lat}: {elevation_value} meters')


# Create a geemap map centered on the location
terrain_map = gfolium.Map(center=[lat, lon], zoom=17)

# load shape of project area
# file_path = os.path.abspath('pp_geometry.geojson')
# with open(file_path) as f:
#    json_shape = json.load(f)

# ee_shape = geojson_to_ee(json_shape)


# polygon_shp = 'pp_geometry.shp'
# polygon = geemap.shp_to_ee(polygon_shp)
# map.addLayer(polygon, {}, 'target_area')

# Use the 'SRTM 30m Digital Elevation Database' dataset to get the elevation
elevation = ee.Image('USGS/SRTMGL1_003').clip(buffer);
slope = ee.Terrain.slope(elevation);
# Calculate aspect. Units are degrees where 0=N, 90=E, 180=S, 270=W.
aspect = ee.Terrain.aspect(elevation);
terrain = ee.Terrain.products(elevation);

vis_params = {
    'min': 700,
    'max': 1600,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

vis_params_slope = {
    'min': 0,
    'max': 20,
    'palette': ['FFFFFF', '000000'],
}

# map.add_ee_layer(elevation, {'min': 70, 'max': 150, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'elevation')
terrain_map.addLayer(elevation, vis_params, 'elevation')
terrain_map.addLayer(slope,{'min': 0, 'max': 20, 'palette': ['FFFFFF', '000000']}, 'slope')
terrain_map.addLayer(aspect,{'min': 0, 'max': 360, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'aspect')
terrain_map.addLayer(terrain.select('hillshade'), {min: 0, max: 255}, 'Hillshade')
terrain_map.add_basemap('SATELLITE')
terrain_map.addLayer(aoi,{'color': 'blue', 'weight': 1, 'opacity': 0.4, 'fillColor' : 'red',  'fillOpacity' : 0}, "shape")
terrain_map.addLayer(center,{'color': 'cyan', 'weight': 1, 'opacity': 0.7}, "center")


# Make pixels with elevation below sea level transparent. -> please implement
# elv_img = srtm.updateMask(srtm.gt(0))

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

terrain_map.add_colorbar_branca(vis_params = vis_params, colors=colors, vmin=vmin, vmax=vmax, layer_name="elevation")
terrain_map.add_colorbar_branca(vis_params = vis_params_slope, colors = vis_params_slope['palette'], vmin= vis_params_slope['min'], vmax= vis_params_slope['max'], layer_name="slope")

# Display the map
terrain_map


In [ ]:
## LANDUSE MAP
# Define the location for which you want to create the map

center = ee.Geometry.Point(lon, lat)
buffer = center.buffer(3000)

landuse_map = gfolium.Map(center=[lat, lon], zoom=17)
dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
landuse_map.addLayer(dataset, {'bands': ['Map']}, 'ESA Land Cover')
landuse_map.add_legend(builtin_legend='ESA_WorldCover')
landuse_map.add_basemap('SATELLITE')
landuse_map.addLayer(aoi,{'color': 'blue', 'weight': 1, 'opacity': 0.4, 'fillColor' : 'red',  'fillOpacity' : 0}, "shape")
landuse_map.addLayer(center,{'color': 'cyan', 'weight': 1, 'opacity': 0.7}, "center")
landuse_map

In [7]:
# download era5 dataset

variable = "wefesiteanalyst"
target_file = 'era5_wefesiteanalyst_'+name+'.nc'
print(target_file)


ds = era5.get_era5_data_from_datespan_and_position(
    variable=variable,
    start_date=start_date, end_date=end_date,
    latitude=lat, longitude=lon,
    target_file=target_file)

era5_wefesiteanalyst_finca_turrialba.nc


2023-01-30 17:57:05,194 INFO Welcome to the CDS
2023-01-30 17:57:05,196 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-01-30 17:57:05,477 INFO Request is queued
2023-01-30 17:57:06,940 INFO Request is running
2023-01-30 18:17:31,484 INFO Request is completed
2023-01-30 18:17:31,505 INFO Downloading request for 6 variables to era5_wefesiteanalyst_finca_turrialba.nc
2023-01-30 18:17:31,514 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data3/adaptor.mars.internal-1675098890.731842-18253-3-01458160-fc48-4216-831a-f61430916fae.nc to era5_wefesiteanalyst_finca_turrialba.nc (143K)
2023-01-30 18:17:33,209 INFO Download rate 84.6K/s                                                                     


In [21]:
# File transformation nc to csv
ds = xr.open_dataset('era5_wefesiteanalyst_'+name+'.nc')
era5_wefe = ds.to_dataframe()
print(era5_wefe)
print(era5_wefe.dtypes)
# convert units
# global horizontal irradiance
era5_wefe['ghi'] = (era5_wefe['ssrd'] / 3600.0)
era5_wefe['t_air'] = era5_wefe['t2m']-273.15
era5_wefe['e'] *=1000
era5_wefe['tp']*=1000

era5_wefe['windspeed'] = calc_sqrt_sum_squares(era5_wefe, 'u10', 'v10')


# date_range = pd.date_range(start='2022-05-01 00:00:00', freq='H', periods=len(era5_climate))

# Set the date range as the index of the DataFrame
# era5_climate.set_index(date_range, inplace=True)


def calc_sqrt_sum_squares(df, col1, col2):
    return np.sqrt(df[col1]**2 + df[col2]**2)

# create new dataframe only consisting out of data for required parameters

columns = ['ghi', 't_air', 'e', 'tp', 'windspeed']
values = era5_wefe[columns].values
ep = pd.DataFrame(values, columns=columns, index = era5_wefe.index)


print(ep)
#print(ep.dtypes)

ep.to_csv("era5_wefesiteanalyst_'+name+'.csv", index=True)


                                                     ssrd         t2m  \
longitude latitude expver time                                          
-83.5     9.75     1      2022-05-01 00:00:00   20877.000  289.848938   
                          2022-05-01 01:00:00       0.000  289.656799   
                          2022-05-01 02:00:00       0.000  289.249207   
                          2022-05-01 03:00:00       0.000  289.780579   
                          2022-05-01 04:00:00       0.000  289.594910   
...                                                   ...         ...   
                   5      2022-11-30 19:00:00  651827.500  291.933838   
                          2022-11-30 20:00:00  486375.625  291.490814   
                          2022-11-30 21:00:00  364889.625  291.495636   
                          2022-11-30 22:00:00  152558.875  290.042572   
                          2022-11-30 23:00:00   44667.125  290.214172   

                                                  

KeyError: 'time'

In [20]:
#era5_wefe = pd.read_csv(r"wefe.csv")
ep.head()

ghi      t_air         e  \
longitude latitude expver time                                                 
-83.5     9.75     1      2022-05-01 00:00:00  5.799167  16.698944 -0.004829   
                          2022-05-01 01:00:00  0.000000  16.506805 -0.000298   
                          2022-05-01 02:00:00  0.000000  16.099213  0.005770   
                          2022-05-01 03:00:00  0.000000  16.630585  0.005076   
                          2022-05-01 04:00:00  0.000000  16.444916  0.000417   

                                                     tp  windspeed  
longitude latitude expver time                                      
-83.5     9.75     1      2022-05-01 00:00:00  2.405666   0.154470  
                          2022-05-01 01:00:00  2.969206   0.505367  
                          2022-05-01 02:00:00  2.260137   0.765288  
                          2022-05-01 03:00:00  1.799162   1.165331  
                          2022-05-01 04:00:00  1.140648   1.027639

In [ ]:
# plot global horizontal irradiance
era5_wefe.loc[:, ['ghi']].plot(title='Irradiance')
plt.xlabel('Time')
plt.ylabel('Irradiance in $W/m^2$')

In [ ]:
# plot total precipitation
era5_wefe.loc[:, ['tp']].plot(title='Total Precipitation')
plt.xlabel('Time')
plt.ylabel('Total Precipitation in $m³/h$')

In [ ]:
# plot temperature
era5_wefe.loc[:, ['t_air']].plot(title='Temperature')
plt.xlabel('Time')
plt.ylabel('Air Temperature in $°C$')